In [14]:
import os
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lit, date_format
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear
from pyspark.sql import types as time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
songs_stage = spark.read.json("s3a://udacity-dend/song_data/A/A/B/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Path does not exist: s3a://udacity-dend/song_data/T/I/L/*.json;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 274, in json
    return self._df(self._jreader.json(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Path does not exist: s3a://udacity-dend/song_data/T/I/L/*.json;'



In [93]:
songs_stage.show(5)
songs_stage.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------------+----------------+---------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|    artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+---------------+---------+---------+------------------+--------------------+----+
|ARKIQCZ1187B9A7C7C|       52.23974|Northampton, Nort...|        -0.88576|        Bauhaus|248.65914|        1|SOSIJKW12A8C1330E3|A God In An Alcov...|   0|
|ARI9DQS1187B995625|       44.93746|  Chippewa Falls, WI|       -91.39251|    Judy Henske|139.78077|        1|SODVVEL12A6D4F9EA0|Good Old Wagon (L...|1964|
|AR2L9A61187B9ADDBC|       52.51607|     Berlin, Germany|        13.37698|Tangerine Dream|492.30322|        1|SOKPKMV12A8C14125E|Catwalk (Black In...|1995|
|ARQTC851187B9B03AF|       39.08166|       Rockville, MD|       

In [74]:
songs_table = songs_stage.select(col('song_id'),
                       col('title'),
                       col('artist_id'),
                       col('year'),
                       col('duration')).orderBy('song_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'desc' is not defined
Traceback (most recent call last):
NameError: name 'desc' is not defined



In [63]:
songs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOAADAD12A8C13D5B0|One Shot (Album V...|ARQTC851187B9B03AF|2005|263.99302|
|SOBXHCE12AB018B6AC|     Arms and Armour|ARV6GHH1187B9AED0D|2008|249.41669|
|SODVVEL12A6D4F9EA0|Good Old Wagon (L...|ARI9DQS1187B995625|1964|139.78077|
+------------------+--------------------+------------------+----+---------+
only showing top 3 rows

In [8]:
songs_table.write.partitionBy('year', 'artist_id').parquet("s3a://spark-output-apk/songs.parquet", 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
artists_table = songs_stage.select(col('artist_id'),
                       col('artist_name').alias('name'),
                       col('artist_location').alias('location'),
                       col('artist_latitude').alias('latitude'),
                       col('artist_longitude').alias('longitude')).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
artists_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+--------+---------+
|         artist_id|              name|          location|latitude|longitude|
+------------------+------------------+------------------+--------+---------+
|ARPIKA31187FB4C233|        The Action|          New York|40.71455|-74.00712|
|AR5AA4Q1187FB4CFBD|    Alisha's Attic|                  |    null|     null|
|ARV6GHH1187B9AED0D|      Sofia Talvik|            SWEDEN|62.19845| 17.55142|
|ARE6D0Y119B8667D9F|Alejandro Santiago|                  |    null|     null|
|ARZD4UW1187B9AB3D2|       Latin Lover|NEW YORK, New York|40.71455|-74.00712|
+------------------+------------------+------------------+--------+---------+
only showing top 5 rows

In [139]:
artists_table.write.parquet(os.path.join("s3a://spark-output-apk/artists.parquet"), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
log_data_raw = spark.read.json("s3a://udacity-dend/log_data/2018/11/2018-11-13-events.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
log_data_stage = log_data_raw.filter(log_data_raw.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
log_data_stage.select('artist', 'song').filter(log_data_stage.artist.contains('B')).show(5)
songs_stage.select('artist_name', 'title').filter(songs_stage.artist_name.contains('Black Keys')).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+
|         artist|                song|
+---------------+--------------------+
|    Dash Berlin|Till The Sky Fall...|
|   Brian Hyland|  Sealed With A Kiss|
| The Black Keys|          Your Touch|
| The Black Keys|     Howlin For You|
|Balkan Beat Box|   Bulgarian  Chicks|
+---------------+--------------------+
only showing top 5 rows

+-----------+-----+
|artist_name|title|
+-----------+-----+
+-----------+-----+

In [91]:
log_data_stage.select('artist').show(5)
log_data_stage.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|        artist|
+--------------+
|            Fu|
|  All Time Low|
|     Nik & Jay|
|Quad City DJ's|
|    Hoobastank|
+--------------+
only showing top 5 rows

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- date: string (nullable = true)

In [69]:
users_table = log_data_stage.select(col('userId').alias('user_id'),
                       col('firstName').alias('firt_name'),
                       col('lastName').alias('last_name'),
                       col('gender'),
                       col('level')).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
users_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+------+-----+
|user_id| firt_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     63|      Ayla|  Johnson|     F| free|
|     37|    Jordan|    Hicks|     F| free|
|      6|   Cecilia|    Owens|     F| free|
|     15|      Lily|     Koch|     F| paid|
|     29|Jacqueline|    Lynch|     F| paid|
+-------+----------+---------+------+-----+
only showing top 5 rows

In [18]:
users_table.write.parquet(os.path.join("s3a://spark-output-apk/users.parquet"), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000))
log_data_stage = log_data_stage.withColumn('timestamp', get_timestamp(log_data_stage.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
log_data_stage = log_data_stage.withColumn('date', get_datetime(log_data_stage.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
log_data_stage.printSchema()
log_data_stage.select('date').show(1, truncate=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- date: string (nullable = true)

+-------------------+
|               date|
+-------------------+
|2018-11-13 00:40:37|
+-------------------+
only showing top 1 row

In [75]:
time_table = log_data_stage.select(col('date').alias('start_time'),
                       hour('date').alias('hour'),
                       dayofmonth('date').alias('day'),
                       weekofyear('date').alias('week'),
                       month('date').alias('month'),
                       year('date').alias('year'),
                       date_format('date','EEEE').alias('weekday')).distinct().orderBy('start_time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
time_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-13 00:40:37|   0| 13|  46|   11|2018|Tuesday|
|2018-11-13 01:12:29|   1| 13|  46|   11|2018|Tuesday|
|2018-11-13 03:19:02|   3| 13|  46|   11|2018|Tuesday|
|2018-11-13 03:51:52|   3| 13|  46|   11|2018|Tuesday|
|2018-11-13 05:00:06|   5| 13|  46|   11|2018|Tuesday|
+-------------------+----+---+----+-----+----+-------+
only showing top 5 rows

In [161]:
time_table.write.partitionBy('year', 'month').parquet("s3a://spark-output-apk/time.parquet", 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o2323.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:172)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:156)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.e

In [102]:
songplays_table = log_data_stage.join(\
                                      songs_stage,\
                                      (log_data_stage.artist == songs_stage.artist_name)\
                                      & (log_data_stage.song == songs_stage.title))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+---------+----+---------+---------------+---------------+----------------+-----------+--------+---------+-------+-----+----+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|timestamp|date|artist_id|artist_latitude|artist_location|artist_longitude|artist_name|duration|num_songs|song_id|title|year|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+---------+----+---------+---------------+---------------+----------------+-----------+--------+---------+-------+-----+----+
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+---------+----+---------+---------------+------------

In [ ]:
songplays_table.write.partitionBy('year', 'month').parquet("s3a://spark-output-apk/songplays.parquet", 'overwrite')